<a href="https://colab.research.google.com/github/Dhyanesh-Panchal/GSR-research-and-stress-detection/blob/master/notebooks/DNN_Statistical_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import HyperParameters

In [4]:
def prepare_statistical_two_class_df():
  data_df = pd.read_csv("https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/statistical_features_over_window/sub_1_w100_g1.csv").drop("Unnamed: 0",axis=1)
  for i in tqdm(range(2,31)):
    df = pd.read_csv(f"https://raw.githubusercontent.com/Dhyanesh-Panchal/GSR-research-and-stress-detection/master/statistical_features_over_window/sub_{i}_w100_g1.csv").drop("Unnamed: 0",axis=1)
    data_df = pd.concat([data_df,df],axis=0)
  filtered_data = data_df[(data_df["arousal"]!=5) & (data_df["valence"]!=5)]
  filtered_data = filtered_data.reset_index().drop("index",axis=1)
  data_2_class = filtered_data.drop(["class_4","arousal","valence"],axis=1)
  data_4_class = filtered_data.drop(["class_2_arousal","arousal","valence"],axis=1)
  data_2_class_encoded = pd.get_dummies(data_2_class,columns=["class_2_arousal"])


  return data_2_class.drop("class_2_arousal",axis=1) , data_2_class["class_2_arousal"]

In [27]:
X,y = prepare_statistical_two_class_df()

  0%|          | 0/29 [00:00<?, ?it/s]

In [30]:
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode the categorical labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)  # Converts 'L' and 'H' to 0 and 1

In [ ]:
train_X,test_X,train_Y,test_Y = train_test_split(X_scaled , y_encoded , test_size=0.2)

In [42]:
# Define the model architecture
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            256, activation="relu", input_shape=train_X.shape[1:]
        ),  # Use full shape for flexibility
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(
            1, activation="sigmoid"
        ),  # Sigmoid for binary classification
    ]
)

In [43]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [44]:
fitting = model.fit(train_X, train_Y,
                    validation_split=0.2,
                    epochs=10)

Epoch 1/10
16610/16610 [==============================] - 75s 4ms/step - loss: 0.6529 - accuracy: 0.5885 - val_loss: 0.6423 - val_accuracy: 0.6012
Epoch 2/10
16610/16610 [==============================] - 77s 5ms/step - loss: 0.6472 - accuracy: 0.5944 - val_loss: 0.6430 - val_accuracy: 0.5920
Epoch 3/10
16610/16610 [==============================] - 75s 5ms/step - loss: 0.6458 - accuracy: 0.5969 - val_loss: 0.6384 - val_accuracy: 0.6136
Epoch 4/10
16610/16610 [==============================] - 77s 5ms/step - loss: 0.6445 - accuracy: 0.6022 - val_loss: 0.6381 - val_accuracy: 0.6149
Epoch 5/10
16610/16610 [==============================] - 74s 4ms/step - loss: 0.6436 - accuracy: 0.6056 - val_loss: 0.6395 - val_accuracy: 0.6118
Epoch 6/10
16610/16610 [==============================] - 76s 5ms/step - loss: 0.6427 - accuracy: 0.6084 - val_loss: 0.6392 - val_accuracy: 0.6124
Epoch 7/10
16610/16610 [==============================] - 75s 5ms/step - loss: 0.6422 - accuracy: 0.6096 - val_loss: 0

In [45]:
model.save_weights('DNN_statistical_clean_WEIGHTS.h5')

In [48]:
# Predict probabilities
probabilities = model.predict(test_X)

# Convert probabilities to class labels
predictions = (probabilities > 0.5).astype(int)

1093/1093 [==============================] - 2s 2ms/step


In [50]:
# Generate the confusion matrix
conf_matrix = confusion_matrix(test_Y, predictions)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[13569  6350]
 [ 6788  8260]]


In [52]:
# Generate the classification report
class_report = classification_report(test_Y, predictions, target_names=['Class 0', 'Class 1'])
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.67      0.68      0.67     19919
     Class 1       0.57      0.55      0.56     15048

    accuracy                           0.62     34967
   macro avg       0.62      0.62      0.62     34967
weighted avg       0.62      0.62      0.62     34967

